# 3. Užduotis

Duomenys iš pirmos užduoties 3) punkto, t.y. a.d. $X$, turinčio binominį skirstinį su
parametrais $k$ ir $p$, t. y.

$$
P(X = m | k, p) = C^k_m p^m {(1 − p)}^{k−m},
$$

dydžio $n = 50$ paprastoji atsitiktinė imtis. Raskite tikslų parametro $p$ pasikliovimo intervalą
naudodami beta skirstinio kvantilius.